# VI. Types, type hierarchy, and multiple dispatch

In Julia types play a central role. We've covered enough so far that you can write Julia code as you would in other languages, but having an understanding of types will give you a better understanding of how Julia works and hopefully help you write better Julia code. 

In Julia, everything has a type. As we've already seen, you can figure out an object's type using the __typeof__ funciton.

In [1]:
typeof(1.3)

Float64

In [2]:
typeof(22)

Int64

In [3]:
typeof("julia")

String

In [4]:
typeof(3//2)

Rational{Int64}

In [5]:
typeof(3 + 2im)

Complex{Int64}

In [6]:
typeof(randn(5, 5))

Matrix{Float64} (alias for Array{Float64, 2})

The curly braces indicate the type is a **parametric** type. So *Array* is a parametric type since it's a type with type parameters (in this case *Float64* and *2*).

You can use the __eltype__ function to determine the element type.

In [7]:
A = rand(5, 5)

5×5 Matrix{Float64}:
 0.632828   0.508423  0.572483  0.448079   0.659855
 0.0141491  0.98914   0.235996  0.928213   0.794259
 0.33415    0.522938  0.813911  0.0112369  0.850961
 0.675744   0.139904  0.944432  0.126913   0.431928
 0.0388984  0.411716  0.917232  0.835462   0.494508

In [8]:
eltype(A)

Float64

In [9]:
t = ("summer", 32, 4.3)

("summer", 32, 4.3)

In [10]:
typeof(t)

Tuple{String, Int64, Float64}

In [11]:
eltype(t)

Any

Because of Julia's emphasis on types you can sometimes run into unexpected problems. Let's generate a 5x5 random array of integers between 1 and 20:

In [12]:
A = rand(1:20, 5, 5)

5×5 Matrix{Int64}:
 18  11   3  20   8
 10  17   2  14   9
 10   1  13  15  17
 10  12   5   7   5
  8  15   4   4   1

As stated before, arrays are mutable:

In [13]:
A[1, 1] = 19

19

In [14]:
A

5×5 Matrix{Int64}:
 19  11   3  20   8
 10  17   2  14   9
 10   1  13  15  17
 10  12   5   7   5
  8  15   4   4   1

Let's now set element A[1,2] equal to 3.2:

In [15]:
A[1, 2] = 3.2

LoadError: InexactError: Int64(3.2)

What caused the error here?

### How types are organized in Julia:

In Julia, types are organized according to a tree like  structure. At the top of the tree is the *Any* type. So all types are subtypes of *Any*. At the bottom of the tree (the leaves of the tree) are __concrete types__, i.e. *Float64*, *Int64*, *UInt32*, etc. A type can have at most one parent, but possibly more than one child.

To get a sense of how the type hierarchy is organized you can use the functions `subtypes` and `supertype`. Let's start with a simple concrete type *Float64* and traverse the tree upwards.

In [16]:
supertype(Float64)

AbstractFloat

So the parent of *Float64* is a type called *AbstractFloat* which in Julia is called an **abstract type**.

In [17]:
supertype(AbstractFloat)

Real

In [18]:
supertype(Real)

Number

In [19]:
supertype(Number)

Any

In [20]:
supertype(Any)

Any

From the above ouptut, we see that *Real*, *Number*, etc. are also **abstract types** (i.e. they can have children). When you create variables in Julia they can only be __concrete types__, i.e. you can not instantiate a variable of type *Real*.

Let's now start with the *Real* type and use the `subtypes` function to go down the tree.

In [21]:
subtypes(Real)

4-element Vector{Any}:
 AbstractFloat
 AbstractIrrational
 Integer
 Rational

So *Real* has four subtypes: *AbstractFloat*, *Integer*, *Irrational*, *Rational*.

In [22]:
subtypes(Integer)

3-element Vector{Any}:
 Bool
 Signed
 Unsigned

In [23]:
subtypes(Signed)

6-element Vector{Any}:
 BigInt
 Int128
 Int16
 Int32
 Int64
 Int8

In [24]:
subtypes(Int64)

Type[]

We can see that *Int64* is a **concrete type**. There are also subtype and supertype operators you can use to ask questions if a type is a supertype or subtype of some other type. Here we test if *Real* is a subtype of *Number*:

In [25]:
Real <: Number

true

Is *Int64* a subtype of *Float64*? Remember these are both concrete types.

In [26]:
Int64 <: Float64

false

Let's test if *Real* is a supertype of *Rational*:

In [27]:
Real >: Rational

true

Is *Int64* a supertype of *Complex*?

In [28]:
Int64 >: Complex

false

Types are important in Julia for many reasons, but perhaps most obviously due to Julia's approach to calling functions known as **multiple dispatch**.

In Julia many versions of the same named function can exist and these different versions are called **methods**. The methods differ in that they have different function signatures. In general, these different methods correspond to a specific type of function behavior determined by the type of inputs.

You can see this with the built-in multiplication function __*__:

In [29]:
*(3, 2.2)

6.6000000000000005

In [30]:
*("string ","input")

"string input"

As we can see the behavior of __*__ is different depending on the input types: for numbers it does multiplication and for strings it does concatenation. You can see all the methods for a function using **methods**:

In [31]:
methods(+)

# 206 methods for generic function "+":
[1] +(x::T, y::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} in Base at int.jl:87
[2] +(x::T, y::T) where T<:Union{Float16, Float32, Float64} in Base at float.jl:383
[3] +(c::Union{UInt16, UInt32, UInt64, UInt8}, x::BigInt) in Base.GMP at gmp.jl:531
[4] +(c::Union{Int16, Int32, Int64, Int8}, x::BigInt) in Base.GMP at gmp.jl:537
[5] +(c::Union{UInt16, UInt32, UInt64, UInt8}, x::BigFloat) in Base.MPFR at mpfr.jl:398
[6] +(c::Union{Int16, Int32, Int64, Int8}, x::BigFloat) in Base.MPFR at mpfr.jl:406
[7] +(c::Union{Float16, Float32, Float64}, x::BigFloat) in Base.MPFR at mpfr.jl:414
[8] +(x::Union{Dates.CompoundPeriod, Dates.Period}) in Dates at /nas/longleaf/rhel8/apps/julia/1.8.0/share/julia/stdlib/v1.8/Dates/src/periods.jl:378
[9] +(x::P, y::P) where P<:Dates.Period in Dates at /nas/longleaf/rhel8/apps/julia/1.8.0/share/julia/stdlib/v1.8/Dates/src/periods.jl:77
[10] +(x::Dates.Period, y::Dates.Period) in Dates at /nas/longleaf/rhel8/apps/julia/1.8.0/share/julia/stdlib/v1.8/Dates/src/periods.jl:367
[11] +(y::Dates.Period, x::Dates.CompoundPeriod) in Dates at /nas/longleaf/rhel8/apps/julia/1.8.0/share/julia/stdlib/v1.8/Dates/src/periods.jl:369
[12] +(y::Dates.Period, x::Dates.TimeType) in Dates at /nas/longleaf/rhel8/apps/julia/1.8.0/share/julia/stdlib/v1.8/Dates/src/arithmetic.jl:85
[13] +(x::Dates.Period, r::AbstractRange{<:Dates.TimeType}) in Dates at /nas/longleaf/rhel8/apps/julia/1.8.0/share/julia/stdlib/v1.8/Dates/src/ranges.jl:63
[14] +(y::Union{Dates.CompoundPeriod, Dates.Period}, x::AbstractArray{<:Dates.TimeType}) in Dates at /nas/longleaf/rhel8/apps/julia/1.8.0/share/julia/stdlib/v1.8/Dates/src/deprecated.jl:14
[15] +(A::LinearAlgebra.SymTridiagonal, B::LinearAlgebra.SymTridiagonal) in LinearAlgebra at /nas/longleaf/rhel8/apps/julia/1.8.0/share/julia/stdlib/v1.8/LinearAlgebra/src/tridiag.jl:207
[16] +(A::LinearAlgebra.SymTridiagonal, B::LinearAlgebra.Symmetric) in LinearAlgebra at /nas/longleaf/rhel8/apps/julia/1.8.0/share/julia/stdlib/v1.8/LinearAlgebra/src/symmetric.jl:485
[17] +(A::LinearAlgebra.SymTridiagonal{var"#s882", V} where {var"#s882"<:Real, V<:AbstractVector{var"#s882"}}, B::LinearAlgebra.Hermitian) in LinearAlgebra at /nas/longleaf/rhel8/apps/julia/1.8.0/share/julia/stdlib/v1.8/LinearAlgebra/src/symmetric.jl:487
[18] +(x::LinearAlgebra.SymTridiagonal, H::LinearAlgebra.UpperHessenberg) in LinearAlgebra at /nas/longleaf/rhel8/apps/julia/1.8.0/share/julia/stdlib/v1.8/LinearAlgebra/src/hessenberg.jl:109
[19] +(A::LinearAlgebra.SymTridiagonal, B::LinearAlgebra.Diagonal) in LinearAlgebra at /nas/longleaf/rhel8/apps/julia/1.8.0/share/julia/stdlib/v1.8/LinearAlgebra/src/special.jl:157
[20] +(A::LinearAlgebra.SymTridiagonal, B::LinearAlgebra.Tridiagonal) in LinearAlgebra at /nas/longleaf/rhel8/apps/julia/1.8.0/share/julia/stdlib/v1.8/LinearAlgebra/src/special.jl:171
[21] +(A::LinearAlgebra.SymTridiagonal, B::LinearAlgebra.Bidiagonal) in LinearAlgebra at /nas/longleaf/rhel8/apps/julia/1.8.0/share/julia/stdlib/v1.8/LinearAlgebra/src/special.jl:225
[22] +(A::LinearAlgebra.SymTridiagonal{var"#s884", V} where {var"#s884"<:Number, V<:AbstractVector{var"#s884"}}, B::LinearAlgebra.UniformScaling) in LinearAlgebra at /nas/longleaf/rhel8/apps/julia/1.8.0/share/julia/stdlib/v1.8/LinearAlgebra/src/special.jl:243
[23] +(A::BitArray, B::BitArray) in Base at bitarray.jl:1184
[24] +(Da::LinearAlgebra.Diagonal, Db::LinearAlgebra.Diagonal) in LinearAlgebra at /nas/longleaf/rhel8/apps/julia/1.8.0/share/julia/stdlib/v1.8/LinearAlgebra/src/diagonal.jl:184
[25] +(D::LinearAlgebra.Diagonal, S::LinearAlgebra.Symmetric) in LinearAlgebra at /nas/longleaf/rhel8/apps/julia/1.8.0/share/julia/stdlib/v1.8/LinearAlgebra/src/diagonal.jl:188
[26] +(D::LinearAlgebra.Diagonal{var"#s884", V} where {var"#s884"<:Real, V<:AbstractVector{var"#s884"}}, H::LinearAlgebra.Hermitian) in LinearAlgebra at /nas/longleaf/rhel8/apps/julia/1.8.0/share/julia/stdlib/v1.8/LinearAlgeb

What Julia does at runtime is call the specialized version of the function (i.e. method) that corresponds to the number of input arguments and types of all input arguments being passed. This process is what is referred to as multiple dispatch.

For your own functions you can incorporate type annotations, e.g.

In [32]:
function PlusTwoSpecific(x::Float64)
    return x + 2
end

PlusTwoSpecific (generic function with 1 method)

In [33]:
function PlusTwoSpecific(x::Int64)
    return x + 2
end

PlusTwoSpecific (generic function with 2 methods)

In [34]:
methods(PlusTwoSpecific)

# 2 methods for generic function "PlusTwoSpecific":
[1] PlusTwoSpecific(x::Float64) in Main at In[32]:1
[2] PlusTwoSpecific(x::Int64) in Main at In[33]:1

However it is worth noting that Julia will do type inference on the argument being passed in. The first time it encounters this argument type it will infer the argument type, then compile and cache this compiled version of the function for the inferred argument type. The next time the function is called with the same argument type Julia will call the specialized compiled version that it has stored in memory.

Therefore type annotations are not always needed and in many cases you can use a generic function signature:

In [35]:
function PlusTwo(x)
    return x + 2
end

PlusTwo (generic function with 1 method)

Type annotations in your functions usually will not improve the performance of your code and you want to avoid overspecialization of your functions. Type annotations are primarily used to guarantee functions behave a specific way for certain input types.

For example, adding two generically works for *Int64* and *Float64* numeric types, so there is no need to create two separate methods of __PlusTwo__ to deal with each of these types. However, what if I wanted my __PlusTwo__ function to work on say a __Date__ type? 

In [36]:
using Dates

myDate = Date(2021,5,21)

2021-05-21

In [37]:
PlusTwo(myDate)

LoadError: MethodError: no method matching +(::Date, ::Int64)
[0mClosest candidates are:
[0m  +(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:591
[0m  +([91m::T[39m, ::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} at int.jl:87
[0m  +(::TimeType, [91m::StridedArray{<:Union{Dates.CompoundPeriod, Period}}[39m) at /nas/longleaf/rhel8/apps/julia/1.8.0/share/julia/stdlib/v1.8/Dates/src/deprecated.jl:18
[0m  ...

In this case, adding two to a __Date__ type doesn't generically work and the behavior of __PlusTwo__ for this input type needs to defined.

In [38]:
function PlusTwo(x::Date)
    return x + Day(2)
end

PlusTwo (generic function with 2 methods)

In [39]:
PlusTwo(myDate)

2021-05-23

# Exercise 6
* Find the supertype of the Bool type.
* Is Int64 subtype of Bool?
* Display all the methods for the "isless" function.

In this lesson we covered:
* Julia types
* Type operations
* Mulitple dispatch